In [1]:
import numpy as np
import scipy as sp
# from keras import layers, models, optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.layers import Conv3D, Dense, Reshape, Add, Input
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam
from keras import backend as K
from keras.utils import to_categorical, multi_gpu_model
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf

from capsulenet import margin_loss
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask

from data import load_data
from plots import (plot_vox,
                   plot_capsnet_rotation_issue,
                   plot_rotation_issue,
                   plot_shaded,
                   plot_dots,
                   plot_recons)
from utils import upsample_classes, stratified_shuffle

/Users/ryan/repos/3d_model_retriever/.env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test), target_names = load_data('./ModelNet10/')
x_train, y_train, x_val, y_val = stratified_shuffle(x_train, y_train, test_size=.1)
x_train, y_train = upsample_classes(x_train, y_train)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [3]:
n_class = y_test.shape[1]
input_shape = (30, 30, 30, 1)

dim_sub_capsule = 16
dim_primary_capsule = 8
n_channels = 4
primary_cap_kernel_size = 9

first_layer_kernel_size = 9
conv_layer_filters = 48


##### If using multiple GPUS ##########
# with tf.device("/cpu:0"):
x = Input(shape=(30, 30, 30, 1))


conv1 = Conv3D(filters=conv_layer_filters, kernel_size=first_layer_kernel_size, strides=1,
               padding='valid', activation='relu', name='conv1')(x)


# primarycaps = PrimaryCap(conv1, dim_capsule=12, n_channels=32,
#                          kernel_size=9, strides=2, padding='valid')

primarycaps = PrimaryCap(conv1, dim_capsule=dim_primary_capsule, n_channels=n_channels,
                         kernel_size=primary_cap_kernel_size, strides=2, padding='valid')

sub_caps = CapsuleLayer(num_capsule=n_class, dim_capsule=dim_sub_capsule,
                         routings=3, name='sub_caps')(primarycaps)

out_caps = Length(name='capsnet')(sub_caps)

y = Input(shape=(n_class,))
masked_by_y = Mask()([sub_caps, y])

masked = Mask()(sub_caps)

decoder = Sequential(name='decoder')
decoder.add(Dense(512, activation='relu',
                  input_dim=dim_sub_capsule*n_class))
# decoder.add(Dense(64, activation='relu'))
decoder.add(Dense(1024, activation='relu'))
decoder.add(Dense(np.prod(input_shape), activation='sigmoid'))

decoder.add(Reshape(target_shape=input_shape, name='out_recon'))

train_model = Model([x, y], [out_caps, decoder(masked_by_y)])


eval_model = Model(x, [out_caps, decoder(masked)])
similarity_model = Model(x, [])

### manipulate model
noise = Input(shape=(n_class, dim_sub_capsule))
noised_sub_caps = Add()([sub_caps, noise])
masked_noised_y = Mask()([noised_sub_caps, y])
manipulate_model = Model([x, y, noise], decoder(masked_noised_y))

Instructions for updating:
dim is deprecated, use axis instead


In [ ]:
NUM_EPOCHS = 5
INIT_LR = 0.001
lam_recon = .04
optimizer = Adam(lr=INIT_LR)

train_model.compile(optimizer,
                    loss=[margin_loss, 'mse'],
                    loss_weights=[1., lam_recon],
                    metrics={'capsnet': 'accuracy'})

checkpointer = ModelCheckpoint(filepath='example.hdf5',
                               verbose=1, save_best_only=True)
tb = TensorBoard(log_dir='./capsnet_final.log/')

# train_model.fit([x_train, y_train], [y_train, x_train],
#                 batch_size=32, epochs=NUM_EPOCHS,
#                 validation_data=[[x_val, y_val], [y_val, x_val]],
# #                 callbacks=[tb, checkpointer])
#                 callbacks=[tb, checkpointer])

Train on 8000 samples, validate on 399 samples
Epoch 1/5
  32/8000 [..............................] - ETA: 11:39:43 - loss: 0.8200 - capsnet_loss: 0.8100 - decoder_loss: 0.2500 - capsnet_acc: 0.0625

In [ ]:
y_pred, x_recon = eval_model.predict(x_test)

print('Test acc:', np.sum(np.argmax(y_pred, 1) == np.argmax(y_test, 1))/y_test.shape[0])